# Solving the thimblerigger

First of all, lets disable logging from the virtual coach created here.

In [192]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In the next step, I optain a VirtualCoach object as an entry point to the NRP platform.
Since I'm working on a local install, we will see a warning message telling us that we have not provided a user name.
This is fine for now.

In [193]:
# log into the virtual coach, update with your credentials
try:
    from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
    vc = VirtualCoach(environment='local')
except ImportError as e:
    print(e)
    print("You have to start this notebook with the command:\
          cle-virtual-coach jupyter notebook")
    raise e

Now I just cluster all imports needed in one global cell so I know where everything is.

In [194]:
import os
import csv
import time
import math
from thread import start_new_thread

import cv2
import rospy
import numpy as np
from cv_bridge import CvBridge
import hbp_nrp_cle.tf_framework as nrp

from gazebo_msgs.msg import ModelState
from std_srvs.srv import Trigger, TriggerResponse
from gazebo_msgs.srv import GetModelState, SetModelState
from std_msgs.msg import UInt32MultiArray, MultiArrayDimension, Float64

import thimblerigger_config as tc

## Moving the robot around the simulation

I need to move the robot around the simulation to find a spot where it has a good view
over the whole challenge setup. Since there may be multiple ways to move around, I define the RobotMover interface,
which does nothing by itself, but has methods to move a model through the simulation.


In [195]:
class RobotMover(object):
    """
    Interface to move an object through the simulation.
    """

    def __init__(self, model_name="robot"):
        """
        param model_name: The name in the gazebo simulation of the object that should move.
        """
        self.model_name = model_name

    def go_to_pose(self, x, y, orientation):
        """
        Moves the object to the given coordinates.
        
        param x: x coordinate to go to.
        param y: y coordinate to go to.
        param orientation: Quaternion orientation vector.
        """
        raise NotImplemented("Please use a subtype of the RobotMover!")

Since this is the perception challenge, I will assume that it is ok to just teleport through the 
simulation. Thus, I build a TeleportRobotMover, which inherits from the RobotMover interface.

In [196]:
class TeleportRobotMover(RobotMover):

    def __init__(self, model_name="robot"):
        """
        param model_name: The name in the gazebo simulation of the object that should be teleported.
        """
        self.get_position = rospy.ServiceProxy("/gazebo/get_model_state", GetModelState)
        self.set_position = rospy.ServiceProxy("/gazebo/set_model_state", SetModelState)
        super(TeleportRobotMover, self).__init__(model_name=model_name)

    def go_to_pose(self, x, y, orientation):
        """
        Teleports the object to the given coordinates.
        
        param x: x coordinate (world coordinates) to go to.
        param y: y coordinate (world coordinates) to go to.
        param orientation: Quaternion orientation vector.
        """
        # Obtain the current pose information, because I do not want
        # to change scale, twist, z-coordinate etc.
        current_robot_pose = self.get_position(self.model_name, "")
        new_state = ModelState()
        new_state.model_name = self.model_name
        new_state.pose = current_robot_pose.pose
        new_state.scale = current_robot_pose.scale
        new_state.twist = current_robot_pose.twist
        new_state.pose.position.x = x
        new_state.pose.position.y = y
        new_state.pose.orientation.x = orientation[1]
        new_state.pose.orientation.y = orientation[2]
        new_state.pose.orientation.z = orientation[3]
        new_state.pose.orientation.w = orientation[0]
        new_state.reference_frame = "world"
        self.set_position(new_state)

## Interacting with the challenge interface

I also need a convenient way to interact with the services provided by the thimblerigger challenge. 
I simply store handles to the ServiceProxies in a wrapper object.

In [197]:
class ChallengeInteractor(object):

    """
    Interface to interact with the thimblerigger challenge.
    """
    
    def __init__(self):
        # Accquire persistent handles to all relevant services
        print("Waiting for show mug service...")
        rospy.wait_for_service(tc.thimblerigger_show_correct_service, 10)
        print("Show mug service available.")
        
        print("Waiting for hide mug service...")
        rospy.wait_for_service(tc.thimblerigger_hide_correct_service, 10)
        print("Hide mug service available.")
        
        print("Waiting for shuffle service...")
        rospy.wait_for_service(tc.thimblerigger_shuffle_service, 10)
        print("Shuffle service available.")
        
        print("Waiting for reset service...")
        rospy.wait_for_service(tc.thimblerigger_reset_service, 10)
        print("Reset service available.")
        
        print("All thimblerigger services found.")                       
        self.show_correct_mug = rospy.ServiceProxy(tc.thimblerigger_show_correct_service, Trigger)
        self.hide_correct_mug = rospy.ServiceProxy(tc.thimblerigger_hide_correct_service, Trigger)
        self.shuffle = rospy.ServiceProxy(tc.thimblerigger_shuffle_service, Trigger)
        self.reset = rospy.ServiceProxy(tc.thimblerigger_reset_service, Trigger)

## Solving the puzzle

Let's create a solver, which actually beats the challenge. For this, we need two transfer functions.
One to visualize what the robot is predicting, and one to make the prediction.

In [198]:
extract_centers = """
@nrp.MapRobotSubscriber("img_msg", Topic("/icub_model/left_eye_camera/image_raw", sensor_msgs.msg.Image))
@nrp.MapVariable("var_center_points", initial_value=[None, None], scope=nrp.GLOBAL)
@nrp.MapVariable("var_estimate", initial_value=None, scope=nrp.GLOBAL)
@nrp.Robot2Neuron()
def extract_centers(t, img_msg, var_center_points, var_estimate):

    if img_msg.value is None:
        return
        
    img = CvBridge().imgmsg_to_cv2(img_msg.value, "bgr8")
    most_vibrant_channel = np.argmax(img, axis=2)
    
    # Filter red 
    img[most_vibrant_channel != 2] = 0
    img[img[:,:,2] < 150] = 0
    red = img[:, :, 2]
    _, thresh = cv2.threshold(red, 150, 255, 0)
    kernel = np.ones((5,5), np.uint8)
    thresh = cv2.erode(thresh, kernel,iterations = 1)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_thresh = 50
    contours = [c for c in contours if cv2.contourArea(c) > contour_thresh]

    moments = [cv2.moments(c) for c in contours]
    centers = sorted([(int(M['m10']/M['m00']), int(M['m01']/M['m00'])) for M in moments])
    center_points = np.array(centers)
    
    if center_points.shape != (3, 2):
        return    
        
    var_center_points.value[0], var_center_points.value[1] = center_points, var_center_points.value[0]
    
    estimate = var_estimate.value
    
    if estimate is not None and var_center_points.value != [None, None]:
        current_centers, old_centers = var_center_points.value
        diff = np.linalg.norm(current_centers-old_centers[estimate], axis=-1)
        var_estimate.value = np.argmin(diff)
    
"""

In [199]:
predict = """

@nrp.MapSpikeSource("prediction", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.prediction[i]), nrp.fixed_frequency)
@nrp.MapVariable("var_estimate", initial_value=None, scope=nrp.GLOBAL)
@nrp.Robot2Neuron()
def predict(t, var_estimate, prediction):
    if var_estimate.value is not None:
        new_rates = [0.] * 3
        new_rates[var_estimate.value] = 100.
        prediction.rate = new_rates
"""

In [200]:
track_green = """
import numpy as np

@nrp.MapRobotSubscriber("img_msg", Topic("/icub_model/left_eye_camera/image_raw", sensor_msgs.msg.Image))
@nrp.MapSpikeSource("green_neurons", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.greens[i]), nrp.dc_source)
@nrp.Robot2Neuron()
def track_green(t, img_msg, green_neurons):

    if img_msg.value is None:
        return
    
    img = CvBridge().imgmsg_to_cv2(img_msg.value, "bgr8")
    green_channel = np.squeeze(img[:, :, 1])
    green_channel = green_channel[120:160, 85:245]
    
    m1 = np.mean(green_channel[:, 0:53])
    m2 = np.mean(green_channel[:, 53: 106])
    m3 = np.mean(green_channel[:, 106:])
    mean_green = np.array([m1, m2, m3])
    mean_green[mean_green < 80.] = 0.
    green_neurons.amplitude = mean_green
    
"""

In [201]:
find_initial_track = """

@nrp.MapVariable("var_estimate", initial_value=None, scope=nrp.GLOBAL)
@nrp.MapSpikeSink("green_neurons", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.greens[i]), nrp.leaky_integrator_alpha)
@nrp.Neuron2Robot()
def find_initial_track(t, var_estimate, green_neurons):

    if var_estimate.value is not None:
        return

    volt = green_neurons.voltage
    one_lifted = np.any(volt > 1.)
    if one_lifted:
        correct_id = np.argmax(volt)
        var_estimate.value = correct_id
"""

In [202]:
class Solver(object):
    
    """
    Solver for the thimblerigger challenge.
    """
    
    def __init__(self, tfs_to_add, shutdown=False):
        """
        param predict_tf: Transfer function that predicts which mug contains the ball.
        param visualize_tf: Transfer function that visualizes the predictions made by predict_tf.
        param shutdown: Shut down the simulation on context exit. Otherwise pause.
        """
        self.sim = vc.launch_experiment('ExDPerceptionChallengeKIT')
        self.tfs = tfs_to_add
        
        self.shutdown = shutdown
        
        self.solved = False
        self.current_view = "front"
        self.robot_mover = None 
        self.challenge = None 


    def set_joints_start(self):
        """
        Moves the robot to a position from which it can overview the challenge setup.
        """
        neck = rospy.Publisher("/robot/neck_pitch/pos", Float64, queue_size=1)
        l_elbow = rospy.Publisher("/robot/l_elbow/pos", Float64, queue_size=1)
        r_elbow = rospy.Publisher("/robot/r_elbow/pos", Float64, queue_size=1)
        
        def lower_gaze_and_arms():
            while not rospy.is_shutdown():
                neck.publish(Float64(-0.8))
                l_elbow.publish(Float64(-25.0))
                r_elbow.publish(Float64(-25.0))
            l_elbow.unregister()
            r_elbow.unregister()
            
        start_new_thread(lower_gaze_and_arms, ())
        # The sleeps are necessary to stop the robot from tumbling over
        # because it teleports and moves at the same time.
        time.sleep(3)
        self.side_look()
        time.sleep(3)
        print("Moved robot to start pose")
        
    def __enter__(self):
        """
        Start the simulation and move the robot to the starting pose.
        Also accquire handles to interact with the challenge.
        """
        self.sim.start()
        self.robot_mover = TeleportRobotMover()
        self.challenge = ChallengeInteractor()
        self.set_joints_start()
        # Robot should be in start pose now
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        Pause the execution of the simulation so the results can be viewed.
        """
        if self.shutdown or exc_type is not None:
            print("Shutting the simulation down...")
            self.sim.stop()
        else:
            print("Pausing the simulation...")
            self.sim.pause()
        
        return False
            

    def front_look(self):
        """
        Moves the robot to the challenge defined start position.
        """
        self.robot_mover.go_to_pose(x=-0.75, y=0., orientation=(0, 0, 1, 0))
        self.current_view = "front"


    def side_look(self):
        """
        Moves the robot to the custom start position.
        """
        orientation = (math.sqrt(0.5), 0, 0, -math.sqrt(0.5))
        self.robot_mover.go_to_pose(x=0.4, y=-0.9, orientation=orientation)
        self.current_view = "side"
        
    def solve(self, interactive=True):
        """
        Solves the currently started challenge.
        
        param interactive: Wait for user input between all steps.
        """
        def maybe_get_input(phrase="continue"):
            if interactive:
                raw_input("Insert 'Enter' to {}.".format(phrase))
                
        if self.solved:
            maybe_get_input("reset the simulation")
            self.sim.start()
            
            # TODO check how this works
            @nrp.MapVariable("estimate", initial_value=None, scope=nrp.GLOBAL)
            def reset_estimate(estimate):
                estimate.value = None
            
            reset_estimate()
            self.challenge.reset()
            
        
        for tf_code in self.tfs.values():
            self.sim.add_transfer_function(tf_code)
      
        time.sleep(2)  # Wait for the TFs to be registered
            
        maybe_get_input(phrase="show the correct mug")        
        self.challenge.show_correct_mug()
        maybe_get_input(phrase="hide the correct mug")
        self.challenge.hide_correct_mug()
        maybe_get_input(phrase="shuffle the mugs")
        self.challenge.shuffle()
        maybe_get_input(phrase="show to correct mug")
        self.challenge.show_correct_mug()   
        self.solved = True
                
        for tf_name in self.tfs.keys():
            self.sim.delete_transfer_function(tf_name)

# Run the solver

Now we simply need to create the solver object within its context.
I recommend watching the frontend in a separate window.
The experiment will pause itself. You can run the solve method multiple times.
Please note that in some cases shuffling the mugs will appear to not work, as a random permutation is chosen.
In some cases, that permutation is the one the mugs are already in.

In [203]:
test = {"extract_centers": extract_centers,
        "track_green": track_green,
        "find_initial_track": find_initial_track,
        "predict": predict}
with Solver(tfs_to_add=test,
            shutdown=False) as solver:
    
    # Pass interactive=False to just run everything at once.
    solver.solve(interactive=True)


KeyboardInterrupt: 